# Train model

Fix conda slow loading https://github.com/pytorch/pytorch/issues/537    

## Tensorboard loading

In [1]:
# get_ipython().system_raw('tensorboard --logdir /tmp/log --host 0.0.0.0 --port 6006 &')

In [2]:
# get_ipython().system_raw('lt --port 6006 >> url.txt 2>&1 &')

In [3]:
# !cat url.txt

In [3]:
from processing_utils import image_manipulations as i_manips
from processing_utils import runtime_logic
from processing_utils import analysis_utils
from processing_utils import load_data
from model.retinanet import model as retina_model

import argparse
import os
import pickle
from copy import deepcopy

import yaml
import torch
import numpy as np
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import datasets, models
from torchvision.transforms import Compose, Normalize, ToTensor, ColorJitter, FiveCrop, CenterCrop, Lambda, RandomCrop, Resize

In [ ]:
print(f'Cuda available: {torch.cuda.is_available()}')

Cuda available: False


## Hyperparameters

In [4]:
seed = 0
torch.cuda.manual_seed(seed)    

run_name = "object_detection_test"
data_root = os.path.join(os.getcwd(), "dummy_data")
img_directory = os.path.join(data_root, "train/")

normalize_params_file = f'outputs/norm_params/{run_name}_norm_params.yaml'
previous_model_state = None # Pickled model state dictionary path

# Can be set to none if TensorboardX isn't installed
tensorboard_outpath = None # f'outputs/tensorboard/{run_name}_log' # Set to none for temp

n_epochs = 25
input_size = 224
init_l_rate = 1e-4
lr_decay = 0.1
lr_decay_epoch = [5, 10, 15]
w_decay = 1e-4

batch_size = 3
num_channels = 3
num_classes = 2

class_weights = torch.Tensor([1, 1])

criterion = nn.CrossEntropyLoss(class_weights)
if torch.cuda.is_available():
    criterion.cuda()

report_results_per_n_batches = {'train':5, 'val':1}
save_interval = 9999

shutdown_after = False

In [6]:
model = retina_model.resnet18(2, pretrained=False)

# model = models.resnet34(pretrained=True)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, num_classes)
# model.avgpool = nn.AdaptiveAvgPool2d(1)

if torch.cuda.is_available():
    criterion = criterion.cuda()
    model = model.cuda()

In [ ]:
# state_dict = torch.load(previous_model_state)
# model.load_state_dict(state_dict)
# if torch.cuda.is_available():
#     model = model.cuda()

## Data Loading

### Get normalize parameters

In [ ]:
# # Get normalize parameters
all_imgs = i_manips.get_images(img_directory)

if os.path.isfile(normalize_params_file):
    stream = open(normalize_params_file, 'r')
    norm_params = yaml.load(stream)
else:
    norm_params = i_manips.get_normalize_params(all_imgs, num_bands=num_channels)
    analysis_utils.write_normalize_values(norm_params, normalize_params_file)

means = norm_params["means"]
sdevs = norm_params["sdevs"]

In [ ]:
train_transforms = Compose([
    ColorJitter(0.05, 0.05, 0.05),
    ToTensor(),
    Normalize([means[0],means[1],means[2]],
              [sdevs[0],sdevs[1],sdevs[2]])
])

val_transforms = Compose([
    ToTensor(),
    Normalize([means[0],means[1],means[2]],
              [sdevs[0],sdevs[1],sdevs[2]])
])   

In [ ]:
## Load training data
train_data = datasets.ImageFolder(os.path.join(data_root, 'train'),
                                  transform=train_transforms)
val_data = datasets.ImageFolder(os.path.join(data_root, 'val'),
                                transform=train_transforms)

train_loader = DataLoader(train_data,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=4)

val_loader = DataLoader(val_data,
                        batch_size=batch_size,
                        shuffle=True,
                        num_workers=4)

classes = deepcopy(train_data.classes)

### Instantiate runtime

In [ ]:
optimizer = optim.RMSprop(model.parameters(), lr=init_l_rate, weight_decay=w_decay)
train_analysis = runtime_logic.AnnotatedImageAnalysis(model,
                                                      classes,
                                                      means,
                                                      sdevs,
                                                      train_loader=train_loader,
                                                      val_loader=val_loader)

In [ ]:
train_analysis.instantiate_loss_tracker('outputs')
train_analysis.loss_tracker.setup_output_storage(run_name)
train_analysis.instantiate_visualiser(tensorboard_outpath)

## Instantialize trainer

In [ ]:
arguments = {# model components
             'run_name':run_name,
             'optimizer':optimizer,
             'criterion':criterion,

             # Hyperparameters
             'n_epochs':n_epochs,
             'batch_size':batch_size,
             'lr_decay':lr_decay,
             'lr_decay_epoch':lr_decay_epoch,
             # 'lr_decay_patience':lr_decay_patience,
             # 'class_weights':class_weights,

             # Saving & Information retrieval
             'report_interval':report_results_per_n_batches,
             'save_interval':save_interval,
    
             'shutdown':shutdown_after,
            }

## Perform training

In [ ]:
train_analysis.train(arguments)

train: [32 out of 82] : 0.8906


In [ ]:
# train_analysis.loss_tracker.save_model(model, 0)